In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
#READING THE INPUT FILE
data=pd.read_excel('C:\Users\18270\Downloads\BlackCoffer\Input.xlsx')[['URL_ID','URL']]

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-4-b34bb4632f32>, line 2)

In [ ]:
data

In [ ]:
import requests
from bs4 import BeautifulSoup
# Function to extract text from a URL
def extract_text_from_url(url):
    # Send GET request to the URL
    response = requests.get(url)
    
    # Parseing the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting the text from the HTML content
    text = soup.get_text()
    
    return text

# Applying the function to each URL in the DataFrame
data['Text'] = data['URL'].apply(extract_text_from_url)

data


In [ ]:
data.Text

In [ ]:
data.info()

In [ ]:
import nltk
import string
nltk.download('punkt')

#EDA

In [ ]:
data.Text=[Text.strip() for Text in data.Text] # remove both the leading and the trailing characters
data.Text=[Text for Text in data.Text if Text] # removes empty strings, because they are considered in Python as False
data[0:10]

In [ ]:
data.Text = data.Text.apply(lambda x :x.lower())          #converting all the data in lower case

In [ ]:
data


In [ ]:
import re
data['Text'] = data['Text'].apply(lambda x: re.sub('[^a-zA-Z .]', ' ', x))   #removing all the characters except ^a-zA-Z . 

In [ ]:
data

In [ ]:
#STOPWORDS

In [ ]:
Auditor=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_Auditor.txt",header=None)
Currencies=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_Currencies.txt",header=None,encoding="ISO-8859-1",sep='|' )
DatesandNumbers=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_DatesandNumbers.txt",header=None)
Generic=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_Generic.txt",header=None)
GenericLong=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_GenericLong.txt",header=None)
Geographic=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_Geographic.txt",header=None)
Names=pd.read_csv("/content/drive/MyDrive/BlackCoffer/StopWords/StopWords_Names.txt",header=None)

In [ ]:
from nltk import word_tokenize               #calling Tokenization function 
data.Text = data.Text.astype(str)
data.Text = data.Text.apply(lambda x:x.split())            #spliting data in words

In [ ]:
data

In [ ]:
#convert all STOPWORDS files to string 
Auditor=Auditor.astype(str)
Currencies=Currencies.astype(str) 
DatesandNumbers=DatesandNumbers.astype(str)
Generic=Generic.astype(str)
GenericLong=GenericLong.astype(str)
Names=Names.astype(str)


In [ ]:
#Convert all stopwords to lower case
Auditor= Auditor.applymap(lambda x:x.lower())
Currencies=Currencies.applymap(lambda x:x.lower())
DatesandNumbers=DatesandNumbers.applymap(lambda x:x.lower())
Generic=Generic.applymap(lambda x:x.lower())
GenericLong=GenericLong.applymap(lambda x:x.lower())
Geographic= Geographic.applymap(lambda x:x.lower())
Names = Names.fillna('').applymap(lambda x: x.lower())
Names= Names.applymap(lambda x:x.lower())

In [ ]:
def clear_stopwords(Text):
      
      Text = [word for word in Text if word not in set(Auditor)]
      Text = [word for word in Text if word not in set(Currencies)]
      Text = [word for word in Text if word not in set(DatesandNumbers)]
      Text = [word for word in Text if word not in set(Generic)]
      Text = [word for word in Text if word not in set(GenericLong)]
      Text = [word for word in Text if word not in set(Geographic)]
      Text = [word for word in Text if word not in set(Names)]
      return Text

In [ ]:
data['Text'] = data['Text'].apply(clear_stopwords)

In [ ]:
data

In [ ]:
data.Text = data.Text.apply(lambda x:" ".join(x))           #join the tokens into sentence again

In [ ]:
data

In [ ]:
#Importing master dictionary 

In [ ]:
positive = pd.read_csv("/content/drive/MyDrive/BlackCoffer/MasterDictionary/positive-words.txt")
negative = pd.read_csv("/content/drive/MyDrive/BlackCoffer/MasterDictionary/negative-words.txt", encoding='ISO-8859-1')


In [ ]:
positive


In [ ]:
negative

In [ ]:
positive = positive.rename(columns={'a+': 'p_word'})                    #Renaming the positive and negative dataframe columns 
negative = negative.rename(columns={'2-faced': 'n_word'})           

In [ ]:
positive.p_word= positive.p_word.str.lower()
positive = positive.apply(clear_stopwords)


In [ ]:
len(positive)

In [ ]:
negative.n_word= negative.n_word.str.lower()
negative = negative.apply(clear_stopwords)


In [ ]:
positive = positive['p_word'].to_list()
negative = negative['n_word'].to_list()

In [ ]:
data.Text

In [ ]:
data.Text = data.Text.astype(str)

In [ ]:
data['tokens'] = data['Text'].apply(lambda x: word_tokenize(x))                      #again tokenizing the data 
data

#Sentimental Analysis

In [ ]:
data['Positive Score'] = 0
data['Negative Score'] = 0
data['Polarity Score'] = 0
data['Subjectivity Score'] = 0



i=0
p_score=0
n_score=0
while(i<len(data)):
  tokens = data['tokens'].iloc[i]                       #POSITIVE SCORE
  for a in tokens:
    if(a in positive):
      p_score+=1
      data['Positive Score'].iloc[i]=p_score
  for b in tokens:                                      #NEGATIVE SCORE
    if(b in negative):
      n_score+=1
      data['Negative Score'].iloc[i]=n_score
  #Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
  Polarity_Score=(p_score-n_score)/((p_score+n_score)+0.000001)
  data['Polarity Score'].iloc[i]=Polarity_Score
    
  #Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
  subjectivity_score=(p_score+n_score)/((len(tokens))+ 0.000001)
  data['Subjectivity Score'].iloc[i]=subjectivity_score
     
  p_score=0
  n_score=0
  i += 1


In [ ]:
data

#Analysis of Readibibility

In [ ]:
data.Text = data.Text.astype(str)
from nltk.tokenize import sent_tokenize
data['sentences'] = data['Text'].apply(lambda x: sent_tokenize(x))                  #tokenizing the sentence in data


In [ ]:
len(data['sentences'].iloc[0])

In [ ]:
#AVERAGE SENTENCE LENGTH
i = 0
while i < len(data):
    words = data['tokens'].iloc[i]
    sent = data['sentences'].iloc[i]
    avg_sent_len = len(words) / len(sent)
    data.at[i, 'Average Sentence Length'] = avg_sent_len
    i += 1



In [ ]:
#Percentage of complex word     AND Complex Word Count
data['Percentage of Complex words']=0
data['Complex Word Count']=0
vowels=['a','e','i','o','u']
count=0
complex_Word_Count=0
i = 0  # initialize i
while(i<len(data)):
 for a in data.tokens.iloc[i]:
   x=re.compile('[es|ed]$')
   if x.match(a):
    count+=0
   else:
     for j in a:
       if(j in vowels ):
         count+=1         
   if(count>2):
    complex_Word_Count+=1
   count=0
 Percentage_of_Complex_words=complex_Word_Count/len(data.tokens.iloc[i])
 data['Percentage of Complex words'].iloc[i]=Percentage_of_Complex_words
 data['Complex Word Count'].iloc[i]= complex_Word_Count
 i += 1  # increment i after each iteration


In [ ]:
#FOG INDEX

In [ ]:
i = 0
while i < len(data):
    fog_index = 0.4 * (data.at[i, 'Average Sentence Length'] + data.at[i, 'Percentage of Complex words'])
    data.at[i, 'Fog Index'] = fog_index
    i += 1



In [ ]:
#AVERAGE NO OF WORDS PER SCENTENCE


In [ ]:
data['avg no of words per sentence'] = 0
avg_length = []
for i in range(0, len(data)):
    for sentence in data['sentences'].iloc[i]:
        words = sentence.split()
        avg_length.append(len(words))

avg_no_of_words_per_sentence = sum(avg_length)/len(avg_length)
data['avg no of words per sentence'] = avg_no_of_words_per_sentence

In [ ]:
#WORD COUNT
data['word count'] = 0
i = 0
while(i < len(data)):
    word_count = len(data['tokens'].iloc[i])
    data.at[i, 'word count'] = word_count
    i += 1

In [ ]:
#SYLLABLE PER WORD
vowels = ['a', 'e', 'i', 'o', 'u']
complex_Word_Count = 0
i = 0
while i < len(data):
    count = 0
    for word in data['tokens'].iloc[i]:
        if word.endswith(('es', 'ed')):
            continue
        for letter in word:
            if letter.lower() in vowels:
                count += 1
    data.at[i, 'Syllable per word'] = count
    i += 1

In [ ]:
#PERSONAL PRONOUNS
pronouns = ['i', 'we', 'my', 'ours', 'us']
i = 0
while i < len(data):
    count = 0
    for word in data['tokens'].iloc[i]:
        if word.lower() in pronouns:
            count += 1
    data.at[i, 'personal pronouns'] = count
    i += 1


In [ ]:
#AVERAGE WORD LENGTH
i = 0
while i < len(data):
    count = 0
    for word in data['tokens'].iloc[i]:
        count += len(word)
    avg_word_length = count / len(data['tokens'].iloc[i])
    data.at[i, 'Average Word Length'] = avg_word_length
    i += 1

In [ ]:
data

In [ ]:
data.drop(['URL_ID','Text','tokens','sentences'],axis=1,inplace=True)

In [ ]:
data
